In [7]:

import numpy as np
import sys
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from sklearn.model_selection import KFold
from copy import deepcopy
import random

import torch
import torch_geometric
from torch_geometric.datasets import Reddit

# node2vec paper
#https://arxiv.org/pdf/1607.00653.pdf

### Load and preprocess data

In [9]:
from load_data import *
dataset_name = "Reddit"
data_dir = "../Data/" + dataset_name


total_graph = load_reddit(data_dir)
#total_graph = load_youtube(data_dir)
#total_graph = load_flickr(data_dir)
#total_graph = load_blogcatalog(data_dir)
#total_graph = load_cora(data_dir)
#total_graph = load_pubmed(data_dir)
print(total_graph['Multioutput'])

0.0
0.09999992246004301
0.19999984492008602
0.29999976738012907
0.39999968984017203
0.49999961230021506
0.5999995347602581
0.6999994572203011
0.7999993796803441
0.8999993021403871
0.9999992246004301
False


In [10]:
## Create 5-fold validation set for NC

NC_5folds = {}
kf = KFold(n_splits=5, shuffle=True)
nodes = np.array([i+1 for i in range(total_graph['N_nodes'])])
for i, (train_index, test_index) in enumerate(kf.split(nodes)):  
    NC_5folds[i] = {"train":nodes[train_index], "test":nodes[test_index]}


#### Create training and testing graphs for LP

In [14]:

# Select 50% of the edges for training, leave remaining for testing.
# Want the remaining graph to still be connected, so we only remove edges if there are several neighbors

def split_graphs(total_graph, directed=False):
    print("splitting graphs")
    n_test_samples = int(total_graph['N_edges']*0.5)
    training_graph_unbalanced = deepcopy(total_graph["edges"])
    test_graph_unbalanced = {i+1:[] for i in range(total_graph['N_nodes'])}
    LP_test_X = [(1,1)]*n_test_samples*2
    LP_test_Y = [0]*n_test_samples*2
    counter = 0

    high_degree_nodes = []
    n_neighbors = {i+1:0 for i in range(total_graph['N_nodes'])}
    for i in range(total_graph['N_nodes']):
        nb_count = len(total_graph['edges'][i+1]) 
        n_neighbors[i+1] = nb_count
        #if nb_count>1:
        #    high_degree_nodes.append(i+1)
    #print(high_degree_nodes[0:10])

    while counter<n_test_samples:
        node1 = random.choice(total_graph['nodes'])
        if n_neighbors[node1]>1:
            node2 = random.choice(training_graph_unbalanced[node1])
            if n_neighbors[node2]>1:
                # Add to test data
                LP_test_X[counter] = (node1, node2)
                LP_test_Y[counter] = 1
                test_graph_unbalanced[node1].append(node2)

                # remove edge from training graph
                training_graph_unbalanced[node1].remove(node2)

                # add/remove reverse edge in case of undirected graphs                 
                if not directed:
                    test_graph_unbalanced[node2].append(node1)
                    training_graph_unbalanced[node2].remove(node1)
                    n_neighbors[node2] -= 1
    
                # decrease neighbor count
                n_neighbors[node1] -= 1
                counter += 1          
                if counter%int(n_test_samples/10)==0:
                    print(counter/n_test_samples)

    return LP_test_X, LP_test_Y, training_graph_unbalanced, test_graph_unbalanced


def balance_test_graph(LP_test_X, LP_test_Y, test_graph_unbalanced, directed=False, reverse_fraction=0.5):
    print("balancing test graph")
    counter = 0
    n_test_samples = int(total_graph['N_edges']*0.5)
    # in case of directed graphs, a fraction of the negative edges are added by reversing true edges
    if directed:
        true_edges = LP_test_X[0:n_test_samples]
        while counter<int(n_test_samples*reverse_fraction):
            true_edge = random.choice(true_edges)
            src = true_edge[0]
            target = true_edge[1]
            if not src in test_graph_unbalanced.get(target):
                LP_test_X[n_test_samples+counter] = (target, src)
                counter += 1
            
            if counter%int(n_test_samples/10)==0:
                print(counter/n_test_samples)

    while counter<n_test_samples:
        node1, node2 = random.sample(total_graph['nodes'], 2)
        if not node1 in test_graph_unbalanced[node2]:
            try:
                LP_test_X[n_test_samples+counter] = (node1, node2)
                LP_test_Y[n_test_samples+counter] = 0
            except:
                LP_test_X.append((node1, node2))
                LP_test_Y.append(0)
                print("appended edge")
            counter += 1
    
        if counter%int(n_test_samples/5)==0:
            print(counter/n_test_samples)
    return LP_test_X, LP_test_Y

# When created the test set, we add remaining edges to the training set
# and add negative edges to balance the training data
def balance_training_graph(training_graph_unbalanced, total_graph, directed=False):
    print("balancing training graph")
    n_test_samples = int(total_graph['N_edges']*0.5)
    LP_train_X = []
    LP_train_Y = []
    added_edges = {i+1:{} for i in range(total_graph['N_nodes'])}
    for node, neighbors in training_graph_unbalanced.items():
        for nb in neighbors:
            if not added_edges[node].get(nb, False):
                added_edges[node][nb] = True
                if not directed:
                    added_edges[nb][node] = True
                LP_train_X.append((node, nb))
                LP_train_Y.append(1)

    n_negative_edges = 0
    while n_negative_edges < n_test_samples:
        node1, node2 = random.sample(total_graph['nodes'], 2)
        if not node1 in training_graph_unbalanced[node2]:
            LP_train_X.append((node1, node2))
            LP_train_Y.append(0)
            n_negative_edges += 1

        if n_negative_edges%int(n_test_samples/10)==0:
            print(n_negative_edges/n_test_samples)
        
    return LP_train_X, LP_train_Y


reverse_fraction = 0
LP_test_X_unb, LP_test_Y_unb, training_graph_unbalanced, test_graph_unbalanced = split_graphs(total_graph, directed=True)
LP_test_X, LP_test_Y = balance_test_graph(LP_test_X_unb, LP_test_Y_unb, test_graph_unbalanced, directed=True, reverse_fraction=reverse_fraction)
LP_train_X, LP_train_Y = balance_training_graph(training_graph_unbalanced, total_graph, directed=True)


splitting graphs
1
0.09999984492007266
0.19999968984014532
0.299999534760218
0.39999937968029065
0.49999922460036333
0.599999069520436
0.6999989144405087
0.7999987593605813
0.899998604280654
0.9999984492007267
balancing test graph
0.1999998621511757
0.3999997243023514
0.5999995864535271
0.7999994486047028
0.9999993107558786
balancing training graph
0.09999984492007266
0.19999968984014532
0.299999534760218
0.39999937968029065
0.49999922460036333
0.599999069520436
0.6999989144405087
0.7999987593605813
0.899998604280654
0.9999984492007267


### Functions for training  

In [18]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''
    def __init__(self):
        self.epoch = 0
       
    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        self.epoch += 1

def compute_pi(total_graph, p, q):
    pi_dict = {i+1:[] for i in range(total_graph['N_nodes'])}
    for i in range(total_graph['N_nodes']):
        neighbors = total_graph['edges'].get(i+1, [])
        n_neighbors = len(neighbors)
        probability_dist = np.ones((n_neighbors+2))
        probability_dist[0:n_neighbors] *= 1/q
        #probability_dist[n_neighbors] = 1     # staying at current node
        probability_dist[n_neighbors+1] = 1/p   # returning to the same node we came from
        norm = 1/p + 1 + n_neighbors/q
        p_normed = probability_dist/norm
        pi_dict[i+1] = p_normed
    return pi_dict 


def alias_sample(prev, current, neighbors, pi_dict):
    n_neighbors = len(neighbors)
    p_normed = pi_dict[current]
    sampled_indx = np.random.choice(n_neighbors+2,  p=p_normed)
    if sampled_indx==n_neighbors:
        return current
    elif sampled_indx==n_neighbors+1:
        return prev
    else:
        return neighbors[sampled_indx]

    

def learn_features(G, dim, walks_per_node, walk_length, context_size, p, q, SGD_epochs):
    pi = compute_pi(G, p, q)
    walks = [[]]*walks_per_node*G['N_nodes']
    c = 0
    for i in range(walks_per_node):
        print(i)
        for node in G["nodes"]:
            walk = node2vec_walk(G, node, walk_length, p, q, pi)
            walks[c] = walk
            c += 1
            if node%int(G["N_nodes"]/10)==0:
                print(node/G['N_nodes'])
 
    f = SDG(walks, context_size, dim, SGD_epochs)
    return f


def node2vec_walk(G, start_node, walk_length, p, q, pi):
    walk = [0]*(walk_length+1)
    walk[0] = start_node
    for i in range(walk_length):
        curr = walk[i]
        if i==0:
            prev = start_node
        else:
            prev = walk[i-1]

        neighbors = G['edges'][curr]
        sample = alias_sample(prev, curr, neighbors, pi)
        walk[i+1] = sample
    return walk
    

def SDG(walks, context_size=10, dim=128, n_epochs=5):
    """Use Word2Vec with SGD to learn embedding based on walks"""
    #sg=1 tells it to use skip-gram algorithm, min_count=0 tells it to not skip "word" that occur only 1 time   
    model = Word2Vec(sentences=walks, vector_size=dim, window=context_size, min_count=0, sg=1, workers=8, epochs=n_epochs, compute_loss=True, callbacks=[EpochLogger()])
    return model

### Train embedding model

In [25]:

# Parameters taken from original node2vec paper:
dim = 128    # should be 128
walks_per_node = 5  #should be 10
walk_length = 40    # should be 80
context_size = 10
# From Khosla et al. these piwere the best performing settings in most cases:
p = 0.25
q = 4
SGD_epochs = 1

USE_PRETRAINED = False
if USE_PRETRAINED:
    embedding_model = Word2Vec.load("../Results/node2vec/{}.model".format(dataset_name))
   #embedding_model = Word2Vec.load("../Results/node2vec/blogcatalog.model")
else:
    embedding_model = learn_features(total_graph, dim, walks_per_node, walk_length, context_size, p, q, SGD_epochs)

    embedding_model.save("../Results/node2vec/{}_longer.model".format(dataset_name))

0
0.09999785375485588
0.19999570750971177
0.29999356126456767
0.39999141501942354
0.4999892687742794
0.5999871225291353
0.6999849762839911
0.7999828300388471
0.8999806837937029
0.9999785375485588
1
0.09999785375485588
0.19999570750971177
0.29999356126456767
0.39999141501942354
0.4999892687742794
0.5999871225291353
0.6999849762839911
0.7999828300388471
0.8999806837937029
0.9999785375485588
2
0.09999785375485588
0.19999570750971177
0.29999356126456767
0.39999141501942354
0.4999892687742794
0.5999871225291353
0.6999849762839911
0.7999828300388471
0.8999806837937029
0.9999785375485588
3
0.09999785375485588
0.19999570750971177
0.29999356126456767
0.39999141501942354
0.4999892687742794
0.5999871225291353
0.6999849762839911
0.7999828300388471
0.8999806837937029
0.9999785375485588
4
0.09999785375485588
0.19999570750971177
0.29999356126456767
0.39999141501942354
0.4999892687742794
0.5999871225291353
0.6999849762839911
0.7999828300388471
0.8999806837937029
0.9999785375485588
Epoch #0 start


### Functions for evaluation tasks

In [35]:
def precision_and_recall(Y_true, Y_pred, nclasses):
    # count true positives and false positives and false negatives
    #nclasses = len(Y_true[0])
    TP_list = [0]*nclasses
    FP_list = [0]*nclasses
    FN_list = [0]*nclasses
    for j in range(nclasses):
       for i, pred in enumerate(Y_pred):
            if pred[j]==1 and Y_true[i][j]==1:
                TP_list[j] += 1
            elif pred[j]==1 and  Y_true[i][j]==0:
                FP_list[j] += 1
            elif pred[j]==0 and Y_true[i][j]==1:
                FN_list[j] += 1 

    return TP_list, FP_list, FN_list

def compute_f1_macro(Y_true, Y_pred, nclasses):
    #nclasses = len(Y_true[0])
    TP_list, FP_list, FN_list = precision_and_recall(Y_true, Y_pred, nclasses)
    f1_scores = [0]*nclasses
    for k in range(nclasses):
        if TP_list[k]==0:
            continue
        f1_scores[k] = TP_list[k]/(TP_list[k]+0.5*(FP_list[k]+FN_list[k])) 
    return np.sum(f1_scores)/nclasses


def compute_f1_micro(Y_true, Y_pred, nclasses):
    TP_list, FP_list, FN_list = precision_and_recall(Y_true, Y_pred, nclasses)
    TP = np.sum(TP_list)
    FP = np.sum(FP_list)
    FN = np.sum(FN_list)
    return TP/(TP + 0.5*(FN+FP))


def compute_accuracy(Y_true, Y_pred, nclasses):
    n_correct = 0
    n_tot = 0
    #nclasses = len(Y_true[0])
    for i, pred in enumerate(Y_pred):
        for j in range(nclasses):
            n_tot += 1
            if pred[j]==Y_true[i][j]:
                n_correct += 1
    return n_correct/n_tot

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def get_edge_representation(fu,fv):
    return sigmoid(np.dot(fu,fv))



### Evaluate NC 

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import LabelEncoder
from  sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler

mb = MultiLabelBinarizer(classes=[i+1 for i in range(total_graph['N_classes'])])
scaler = StandardScaler()

def onehot(y, nclasses):
    Y = np.zeros((y.shape[0], nclasses), dtype=int)
    for i in range(y.shape[0]):
        c = y[i]
        Y[i,c-1] =  1
    return Y

f1_macro_list = []
f1_micro_list = []
accuracy_scores_list = []
# 5-fold cross validation
for i in range(5):
    print(i)
    training_nodes = NC_5folds[i]['train']
    test_nodes = NC_5folds[i]['test']
    X_train = scaler.fit_transform(np.array([embedding_model.wv[node] for node in training_nodes], dtype=object))
    X_test = scaler.fit_transform(np.array([embedding_model.wv[node] for node in test_nodes], dtype=object))
    # For the datasets that only have one one label per node, it gives better results to not use multioutputclassifier
    if not total_graph['Multioutput']:
        Y_train_sequence = np.array([total_graph['groups'][node][0]  for node in training_nodes],dtype=int)
        Y_test_sequence = np.array([total_graph['groups'][node][0] for node in test_nodes], dtype=int)
        log_reg = LogisticRegression(multi_class="ovr", max_iter=200)
        Y_train = Y_train_sequence
        Y_test = Y_test_sequence
        log_reg.fit(X_train, Y_train)
        Y_pred = log_reg.predict(X_test)
        Y_pred = onehot(Y_pred, total_graph['N_classes'])
        Y_test = onehot(Y_test, total_graph['N_classes'])
    else:
        Y_train_sequence = np.array([total_graph['groups'][node]  for node in training_nodes],dtype=int)
        Y_test_sequence = np.array([total_graph['groups'][node] for node in test_nodes], dtype=int)
        Y_train = mb.fit_transform(Y_train_sequence)
        Y_test = mb.fit_transform(Y_test_sequence)
        log_reg = MultiOutputClassifier(LogisticRegression(multi_class="ovr"))
        log_reg.fit(X_train, Y_train)
        Y_pred = log_reg.predict(X_test)
        
    acc = compute_accuracy(Y_test, Y_pred, total_graph['N_classes'])
    f1_macro = compute_f1_macro(Y_test, Y_pred, total_graph['N_classes'])
    f1_micro = compute_f1_micro(Y_test, Y_pred,total_graph['N_classes'])
    accuracy_scores_list.append(acc)
    f1_macro_list.append(f1_macro)
    f1_micro_list.append(f1_micro)
    print(f1_macro, f1_micro)
    
print(np.mean(f1_micro_list))
print(np.mean(f1_macro_list))

0
0.0059193137467269475 0.12091945141974117
1
0.005506062599148393 0.11986779129912219
2
0.005631555105599174 0.11950292962462172
3
0.005710089912445558 0.12209988624900736
4
0.005572340452583576 0.12089798896829997
0.1206576095121585
0.00566787236330073


### Evaluate LP

In [54]:
Y_train = LP_train_Y
Y_test = LP_test_Y

from sklearn.metrics import roc_auc_score

# build representation of edge datasets using inner product of the representation of the two nodes
X_train = np.zeros((len(LP_train_X), 1))
for i, edge in enumerate(LP_train_X):
    u = edge[0]
    v = edge[1]
    X_train[i] = get_edge_representation(embedding_model.wv[u], embedding_model.wv[v])
X_test = np.zeros((len(LP_test_X), 1))
for i, edge in enumerate(LP_test_X):
    u = edge[0]
    v = edge[1]
    X_test[i] = get_edge_representation(embedding_model.wv[u], embedding_model.wv[v])
    
print("fit model")
classifier = LogisticRegression()
classifier.fit(X_train, Y_train)
Y_probs = classifier.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(Y_test, Y_probs)
print(roc_auc)
  

fit model
0.9382635986742481


### Save results

In [55]:


with open("../Results/node2vec/{}_metrics{}.csv".format(dataset_name, reverse_fraction), "w") as file:
    settings_str = "Results for Node2vec embedding generated with p={}, q={}, walk length={}, walks per node={}, sgd_epochs={}\n".format(p,q,
    walk_length, walks_per_node, SGD_epochs)
    file.write(settings_str)
    #header = "Dataset; F1 macro; F1 micro; ROC-AUC \n"
    header = "Dataset; F1 macro; F1 micro; ROC-AUC_{} \n".format(reverse_fraction)
    file.write(header)
    data_row = "{dataset};{f1mac};{f1mic};{roc}".format(dataset=dataset_name, f1mac=np.mean(f1_macro_list), f1mic=np.mean(f1_micro_list), roc=None)
    file.write(data_row)

